In [35]:
import requests
from pprint import pprint
import json 
import codecs


endpoint_diffbot = "http://kg.diffbot.com/kg/dql_endpoint"


def get_token():
    with open("../.diffbot-token", "r") as f:
        return f.read().strip()


def query_and_save(query, output_fpath):
    data = {
        "token": get_token(),
        "query": query,
        "type": "query"}
    r = requests.get(endpoint_diffbot, params=data)

    with codecs.open(output_fpath, "w", "utf-8") as out:   
        out.write(json.dumps(json.loads(r.content)))
    print(output_fpath)
    

query_and_save(
    query='type:AdministrativeArea',
    output_fpath="data/aa.json")
    
    
query_and_save(
    query='type:Organization',
    output_fpath="data/organizations.json")


query_and_save(
    query='type:Person',
    output_fpath="data/persons.json")


query_and_save(
    query='type:LocalBusiness',
    output_fpath="data/business.json")

    
query_and_save(
    query='type:Person name:"Alexander Panchenko"',
    output_fpath="data/ap.json")


query_and_save(
    query='type:Person employments.employer.name:"Diffbot"',
    output_fpath="data/diffbot-employees.json")


query_and_save(
    query='type:Person employments.{title:"CEO" employer.name:"Diffbot"}',
    output_fpath="data/diffbot-ceo.json")

query_and_save(
    query='type:Person employments.{employer.name:"Diffbot" isCurrent:true}',
    output_fpath="data/diffbot-current-employees.json")



data/aa.json
data/organizations.json
data/persons.json
data/business.json
data/ap.json
data/diffbot-employees.json
data/diffbot-ceo.json
data/diffbot-current-employees.json


In [15]:
json.dumps??